In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

In [2]:
busfile = pd.read_csv("bus.csv")
genfile = pd.read_csv("gen.csv")
loadfile = pd.read_csv("load.csv")
resfile = pd.read_csv("res.csv")
PTDFfile = pd.read_csv("PTDF.csv")
linefile = pd.read_csv("line.csv")

genfile.set_index('genname', inplace = True)
loadfile.set_index('hour', inplace = True)
resfile.set_index('hour', inplace = True)
PTDFfile.set_index('line', inplace = True)
linefile.set_index("line", inplace = True)

loadfile['all'] = loadfile.sum(axis=1)

In [3]:
OriginalProblem = Model("origin")

Using license file /Users/gabriel79/gurobi.lic
Academic license - for non-commercial use only


In [4]:
buslist = list(busfile.busname)
linelist = list(PTDFfile.index)
genlist = list(genfile.index)
reslist = ['PV1']
nonreslist = list(set(genlist) - set(reslist))
hourlist = list(loadfile.index)

In [5]:
Pgen = {}
Pk1 = {}
Pk2 = {}
Pk3 = {}
Pk4 = {}
Igen = {}
Ygen = {}
Zgen = {}
GenCost = {}
for j in hourlist:
    GenCost[j] = OriginalProblem.addVar(name = "Generation Cost at hour "+str(j+1))
    for i in nonreslist:
        Igen[i,j] = OriginalProblem.addVar(name = "Commitment of "+i+" at hour "+str(j+1), vtype = GRB.BINARY)
        Ygen[i,j] = OriginalProblem.addVar(name = "Start-up of "+i+" at hour "+str(j+1), vtype = GRB.BINARY)
        Zgen[i,j] = OriginalProblem.addVar(name = "Shut-dn of "+i+" at hour "+str(j+1), vtype = GRB.BINARY)
        Pgen[i,j] = OriginalProblem.addVar(name = "Power of "+i+" at hour "+str(j+1), lb = genfile.loc[i,'Pmin'], ub = genfile.loc[i,'Pmax'])
        Pk1[i,j] = OriginalProblem.addVar(name = "Power Segment 1 of "+i+" at hour "+str(j+1), lb = 0, ub = genfile.loc[i,'SP1'])
        Pk2[i,j] = OriginalProblem.addVar(name = "Power Segment 2 of "+i+" at hour "+str(j+1), lb = 0, ub = genfile.loc[i,'SP2'] - genfile.loc[i,'SP1'])
        Pk3[i,j] = OriginalProblem.addVar(name = "Power Segment 3 of "+i+" at hour "+str(j+1), lb = 0, ub = genfile.loc[i,'SP3'] - genfile.loc[i,'SP2'])
        Pk4[i,j] = OriginalProblem.addVar(name = "Power Segment 4 of "+i+" at hour "+str(j+1), lb = 0, ub = genfile.loc[i,'SP4'] - genfile.loc[i,'SP3'])
        
    for i in reslist:
        Pgen[i,j] = OriginalProblem.addVar(name = "Power of "+i+" at hour "+str(j+1), lb = genfile.loc[i,'Pmin'], ub = resfile.loc[j,i])
        

In [6]:
# Generator Constraints
TSP = len(hourlist)
for j in hourlist:
    for i in nonreslist:
        OriginalProblem.addConstr( Pgen[i,j] == Igen[i,j] * genfile.loc[i,'Pmin'] + \
                                   Pk1[i,j] + Pk2[i,j] + Pk3[i,j] + Pk4[i,j], 'Power Segment Summation')
        OriginalProblem.addConstr( Ygen[i,j] + Zgen[i,j] <= 1 + 1e-3, \
                                   'Binary Exclusiveness Constraint for '+str(i)) 
        OriginalProblem.addConstr( Pgen[i,j] >= genfile.loc[i,'Pmin'] * Igen[i,j], \
                                   'Down Gen. Capacity Constraint for '+str(i)) 
        OriginalProblem.addConstr( Pgen[i,j] <= genfile.loc[i,'Pmax'] * Igen[i,j], \
                                   'Up Gen. Capacity Constraint for '+str(i)) 
        
        if j == 1:
            OriginalProblem.addConstr( Pgen[i,j] - genfile.loc[i,'IntPow'] <= genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Up Constraint for '+str(i))   
            OriginalProblem.addConstr( Pgen[i,j] - genfile.loc[i,'IntPow'] >= -genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Down Constraint for '+str(i)) 
            OriginalProblem.addConstr( Ygen[i,j] - Zgen[i,j] == Igen[i,j] - genfile.loc[i,'Status'], \
                                    'Startup/Shutdn Commitment Constraint for '+str(i)) 
        else:
            OriginalProblem.addConstr( Pgen[i,j] - Pgen[i,j-1] <= genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Up Constraint for '+str(i))
            OriginalProblem.addConstr( Pgen[i,j] - Pgen[i,j-1] >= -genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Down Constraint for '+str(i))
            OriginalProblem.addConstr( Ygen[i,j] - Zgen[i,j] == Igen[i,j] - Igen[i,j-1], \
                                    'Startup/Shutdn Commitment Constraint for '+str(i)) 
            
        if genfile.loc[i,'minON'] < 1 + 1e-3:
            continue
            
        if genfile.loc[i,'Status'] == 1 and genfile.loc[i,'minON'] > genfile.loc[i,'IntHour']:
            if j < genfile.loc[i,'minON'] - genfile.loc[i,'IntHour']:
                OriginalProblem.addConstr( Igen[i,j] == 1, \
                                          'Gen MinON/OFF Constraint') 
            elif j < TSP - genfile.loc[i,'minON'] + 1:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, j+int(genfile.loc[i,'minON']))) >= genfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')            
        elif genfile.loc[i,'Status'] == 1 and genfile.loc[i,'minON'] <= genfile.loc[i,'IntHour']:
            if j > TSP-genfile.loc[i,'minON']+1:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, j+int(genfile.loc[i,'minON']))) >= genfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint') 
        elif genfile.loc[i,'Status'] == 0 and genfile.loc[i,'minOFF'] <= genfile.loc[i,'IntHour']:
            if j > TSP-genfile.loc[i,'minON']+1:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, j+int(genfile.loc[i,'minON']))) >= genfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
        elif genfile.loc[i,'Status'] == 0 and genfile.loc[i,'minOFF'] > genfile.loc[i,'IntHour']:
            if j < genfile.loc[i,'minOFF'] - genfile.loc[i,'IntHour']:
                OriginalProblem.addConstr( Igen[i,j] == 0, \
                                          'Gen MinON/OFF Constraint') 
            elif j > TSP - genfile.loc[i,'minON'] + 1:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, genfile.loc[i,'minON'])) >= genfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
                
        if genfile.loc[i,'minOFF'] < 1 + 1e-3:
            continue
            
        if genfile.loc[i,'Status'] == 1 and genfile.loc[i,'minON'] > genfile.loc[i,'IntHour']:
            if j < genfile.loc[i,'minON'] - genfile.loc[i,'IntHour']:
                OriginalProblem.addConstr( Igen[i,j] == 1, \
                                          'Gen MinON/OFF Constraint') 
            elif j < TSP - genfile.loc[i,'minOFF'] + 1:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(genfile.loc[i,'minOFF']))) >= genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')            
        elif genfile.loc[i,'Status'] == 1 and genfile.loc[i,'minON'] <= genfile.loc[i,'IntHour']:
            if j > TSP-genfile.loc[i,'minOFF']+1:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(genfile.loc[i,'minOFF']))) >= genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
        elif genfile.loc[i,'Status'] == 0 and genfile.loc[i,'minOFF'] <= genfile.loc[i,'IntHour']:
            if j > TSP-genfile.loc[i,'minOFF']+1:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(genfile.loc[i,'minOFF']))) >= genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
        elif genfile.loc[i,'Status'] == 0 and genfile.loc[i,'minOFF'] > genfile.loc[i,'IntHour']:
            if j < genfile.loc[i,'minOFF'] - genfile.loc[i,'IntHour']:
                OriginalProblem.addConstr( Igen[i,j] == 0, \
                                          'Gen MinON/OFF Constraint') 
            elif j > TSP - genfile.loc[i,'minOFF'] + 1:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
            else:
                OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(genfile.loc[i,'minOFF']))) >= genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')  


In [7]:
for j in hourlist:
    OriginalProblem.addConstr( GenCost[j] == quicksum(Ygen[i,j] * genfile.loc[i,'SUCost'] \
                                          + Zgen[i,j] * genfile.loc[i,'SDCost'] \
                                          + Igen[i,j] * genfile.loc[i,'NOLOADCOST'] * genfile.loc[i,'Pmin'] \
                                          + Pk1[i,j] * genfile.loc[i,'k1'] + Pk2[i,j] * genfile.loc[i,'k2'] \
                                          + Pk3[i,j] * genfile.loc[i,'k3'] + Pk4[i,j] * genfile.loc[i,'k4'] for i in nonreslist), \
                                       'Generation Cost Constraint')
    
    OriginalProblem.addConstr( quicksum(Pgen[g,j] for g in genlist) == loadfile.loc[j,'all'], \
                                'Power Balance Constraint') 
    
    for i in linelist:
        OriginalProblem.addConstr( quicksum(PTDFfile.loc[i,genfile.loc[g,'busname']] * Pgen[g,j] for g in genlist) \
                                  - quicksum(PTDFfile.loc[i,d] * loadfile.loc[j,d] for d in buslist) <= linefile.loc[i,'FlowLim'], \
                                          'DC Power Flow Constraint') 
        OriginalProblem.addConstr( quicksum(PTDFfile.loc[i,genfile.loc[g,'busname']] * Pgen[g,j] for g in genlist) \
                                  - quicksum(PTDFfile.loc[i,d] * loadfile.loc[j,d] for d in buslist) >= -linefile.loc[i,'FlowLim'], \
                                          'DC Power Flow Constraint')  
        

In [8]:
OriginalProblem.setObjective( quicksum(GenCost[j] for j in hourlist), GRB.MINIMIZE)

In [9]:
OriginalProblem.update()

OriginalProblem.setParam(GRB.Param.Threads, 8)
OriginalProblem.optimize()

Changed value of parameter Threads to 8
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 2064 rows, 816 columns and 7974 nonzeros
Model fingerprint: 0x20f71d78
Variable types: 528 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [5e-03, 6e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 559498.56978
Presolve removed 1921 rows and 711 columns
Presolve time: 0.00s
Presolved: 143 rows, 105 columns, 413 nonzeros
Variable types: 105 continuous, 0 integer (0 binary)

Root relaxation: objective 4.390364e+05, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    439036.42741 439036.427  0.00%     -    0s

Explored 0 nodes (36 simplex iterations) i

In [10]:
if OriginalProblem.status == GRB.Status.OPTIMAL:
    solution = OriginalProblem.getAttr('x', Pgen)

In [11]:
for g in genlist:
    for h in hourlist:
        print(solution[g,h])
        
    print('\n\n')

300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0
300.0



434.6338085921911
411.56711119117386
396.8186459489095
397.16082825175073
423.17561252797145
492.8437059238865
519.7551801356458
580.8696479221891
622.2718103404039
654.0904453069732
665.8339206518808
655.5073497806301
640.9801350292888
624.8490520989806
598.8911709057827
591.3093028692986
624.5602303226206
735.6537065014529
866.2843819832821
841.7376529126883
775.3064316169086
681.6797992103233
570.3606119473772
473.77944878824565



172.1839614078089
149.461196808826
134.93235405109044
135.26994074824933
175.09192447202824
286.3057560761138
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
562.8762393167187
587.4094230873117
507.7740303830916
415.544278789677
305.8857720526229
210.74521921175432



1200.0
1200.0
1200.0
1200.0
1180.0
1120.0
1060.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1060.0
1120.0


In [2]:
# First we run the original problem as an ordinary MILP problem and solve directly by Gurobi.
# Test Instance: 6-bus Power Market's Unit Commitment (6BUC)

class Create_UC_Problem:
    
    def __init__(self, directory, genfile, linefile, loadfile, busfile, resfile, PTDFfile, ProblemName):
        
        self.genfile = pd.read_csv(directory + genfile)
        self.linefile = pd.read_csv(directory + linefile)
        self.loadfile = pd.read_csv(directory + loadfile)
        self.busfile = pd.read_csv(directory + busfile)
        self.resfile = pd.read_csv(directory + resfile)
        self.PTDFfile = pd.read_csv(directory + PTDFfile)
        self.ProblemName = ProblemName

        self.genfile.set_index('genname', inplace = True)
        self.loadfile.set_index('hour', inplace = True)
        self.resfile.set_index('hour', inplace = True)
        self.PTDFfile.set_index('line', inplace = True)
        self.linefile.set_index("line", inplace = True)
        
    def create_instance(self):
        
        buslist = list(self.busfile.busname)
        linelist = list(self.PTDFfile.index)
        genlist = list(self.genfile.index)
        reslist = ['PV1']
        nonreslist = list(set(genlist) - set(reslist))
        hourlist = list(self.loadfile.index)
        
        self.loadfile['all'] = self.loadfile.sum(axis=1)
        
        OriginalProblem = Model(self.ProblemName)
        
        Pgen = {}
        Pk1 = {}
        Pk2 = {}
        Pk3 = {}
        Pk4 = {}
        Igen = {}
        Ygen = {}
        Zgen = {}
        GenCost = {}
        for j in hourlist:
            GenCost[j] = OriginalProblem.addVar(name = "Generation Cost at hour "+str(j+1))
            for i in nonreslist:
                Igen[i,j] = OriginalProblem.addVar(name = "Commitment of "+i+" at hour "+str(j+1), vtype = GRB.BINARY)
                Ygen[i,j] = OriginalProblem.addVar(name = "Start-up of "+i+" at hour "+str(j+1), vtype = GRB.BINARY)
                Zgen[i,j] = OriginalProblem.addVar(name = "Shut-dn of "+i+" at hour "+str(j+1), vtype = GRB.BINARY)
                Pgen[i,j] = OriginalProblem.addVar(name = "Power of "+i+" at hour "+str(j+1), lb = self.genfile.loc[i,'Pmin'], ub = self.genfile.loc[i,'Pmax'])
                Pk1[i,j] = OriginalProblem.addVar(name = "Power Segment 1 of "+i+" at hour "+str(j+1), lb = 0, ub = self.genfile.loc[i,'SP1'])
                Pk2[i,j] = OriginalProblem.addVar(name = "Power Segment 2 of "+i+" at hour "+str(j+1), lb = 0, ub = self.genfile.loc[i,'SP2'] - self.genfile.loc[i,'SP1'])
                Pk3[i,j] = OriginalProblem.addVar(name = "Power Segment 3 of "+i+" at hour "+str(j+1), lb = 0, ub = self.genfile.loc[i,'SP3'] - self.genfile.loc[i,'SP2'])
                Pk4[i,j] = OriginalProblem.addVar(name = "Power Segment 4 of "+i+" at hour "+str(j+1), lb = 0, ub = self.genfile.loc[i,'SP4'] - self.genfile.loc[i,'SP3'])
        
            for i in reslist:
                Pgen[i,j] = OriginalProblem.addVar(name = "Power of "+i+" at hour "+str(j+1), lb = self.genfile.loc[i,'Pmin'], ub = self.resfile.loc[j,i])
        
        
        TSP = len(hourlist)
        for j in hourlist:
            for i in nonreslist:
                OriginalProblem.addConstr( Pgen[i,j] == Igen[i,j] * self.genfile.loc[i,'Pmin'] + \
                                   Pk1[i,j] + Pk2[i,j] + Pk3[i,j] + Pk4[i,j], 'Power Segment Summation')
                OriginalProblem.addConstr( Ygen[i,j] + Zgen[i,j] <= 1 + 1e-3, \
                                   'Binary Exclusiveness Constraint for '+str(i)) 
                OriginalProblem.addConstr( Pgen[i,j] >= self.genfile.loc[i,'Pmin'] * Igen[i,j], \
                                   'Down Gen. Capacity Constraint for '+str(i)) 
                OriginalProblem.addConstr( Pgen[i,j] <= self.genfile.loc[i,'Pmax'] * Igen[i,j], \
                                   'Up Gen. Capacity Constraint for '+str(i)) 
        
            if j == 1:
                OriginalProblem.addConstr( Pgen[i,j] - self.genfile.loc[i,'IntPow'] <= self.genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Up Constraint for '+str(i))   
                OriginalProblem.addConstr( Pgen[i,j] - self.genfile.loc[i,'IntPow'] >= -self.genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Down Constraint for '+str(i)) 
                OriginalProblem.addConstr( Ygen[i,j] - Zgen[i,j] == Igen[i,j] - self.genfile.loc[i,'Status'], \
                                    'Startup/Shutdn Commitment Constraint for '+str(i)) 
            else:
                OriginalProblem.addConstr( Pgen[i,j] - Pgen[i,j-1] <= self.genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Up Constraint for '+str(i))
                OriginalProblem.addConstr( Pgen[i,j] - Pgen[i,j-1] >= -self.genfile.loc[i,'RAMP_RATE'] * 60, \
                                    'Ramping Down Constraint for '+str(i))
                OriginalProblem.addConstr( Ygen[i,j] - Zgen[i,j] == Igen[i,j] - Igen[i,j-1], \
                                    'Startup/Shutdn Commitment Constraint for '+str(i)) 
            
            if self.genfile.loc[i,'minON'] < 1 + 1e-3:
                continue
            
            if self.genfile.loc[i,'Status'] == 1 and self.genfile.loc[i,'minON'] > self.genfile.loc[i,'IntHour']:
                if j < self.genfile.loc[i,'minON'] - self.genfile.loc[i,'IntHour']:
                    OriginalProblem.addConstr( Igen[i,j] == 1, \
                                          'Gen MinON/OFF Constraint') 
                elif j < TSP - self.genfile.loc[i,'minON'] + 1:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, j+int(self.genfile.loc[i,'minON']))) >= gself.enfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')            
            elif self.genfile.loc[i,'Status'] == 1 and self.genfile.loc[i,'minON'] <= self.genfile.loc[i,'IntHour']:
                if j > TSP-self.genfile.loc[i,'minON']+1:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, j+int(self.genfile.loc[i,'minON']))) >= self.genfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint') 
            elif self.genfile.loc[i,'Status'] == 0 and self.genfile.loc[i,'minOFF'] <= self.genfile.loc[i,'IntHour']:
                if j > TSP-self.genfile.loc[i,'minON']+1:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, j+int(self.genfile.loc[i,'minON']))) >= self.genfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
            elif self.genfile.loc[i,'Status'] == 0 and self.genfile.loc[i,'minOFF'] > self.genfile.loc[i,'IntHour']:
                if j < self.genfile.loc[i,'minOFF'] - self.genfile.loc[i,'IntHour']:
                    OriginalProblem.addConstr( Igen[i,j] == 0, \
                                          'Gen MinON/OFF Constraint') 
                elif j > TSP - self.genfile.loc[i,'minON'] + 1:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(Igen[i,k] for k in range(j, self.genfile.loc[i,'minON'])) >= self.genfile.loc[i,'minON'] * Ygen[i,j], \
                                          'Gen MinON/OFF Constraint')
                
            if self.genfile.loc[i,'minOFF'] < 1 + 1e-3:
                continue
            
            if self.genfile.loc[i,'Status'] == 1 and self.genfile.loc[i,'minON'] > self.genfile.loc[i,'IntHour']:
                if j < self.genfile.loc[i,'minON'] - self.genfile.loc[i,'IntHour']:
                    OriginalProblem.addConstr( Igen[i,j] == 1, \
                                          'Gen MinON/OFF Constraint') 
                elif j < TSP - self.genfile.loc[i,'minOFF'] + 1:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(self.genfile.loc[i,'minOFF']))) >= self.genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')            
            elif self.genfile.loc[i,'Status'] == 1 and self.genfile.loc[i,'minON'] <= self.genfile.loc[i,'IntHour']:
                if j > TSP-self.genfile.loc[i,'minOFF']+1:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(self.genfile.loc[i,'minOFF']))) >= self.genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
            elif self.genfile.loc[i,'Status'] == 0 and self.genfile.loc[i,'minOFF'] <= self.genfile.loc[i,'IntHour']:
                if j > TSP-genfile.loc[i,'minOFF']+1:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(self.genfile.loc[i,'minOFF']))) >= self.genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
            elif self.genfile.loc[i,'Status'] == 0 and self.genfile.loc[i,'minOFF'] > self.genfile.loc[i,'IntHour']:
                if j < self.genfile.loc[i,'minOFF'] - self.genfile.loc[i,'IntHour']:
                    OriginalProblem.addConstr( Igen[i,j] == 0, \
                                          'Gen MinON/OFF Constraint') 
                elif j > TSP - self.genfile.loc[i,'minOFF'] + 1:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, TSP+1)) >= (TSP-j+1) * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')
                else:
                    OriginalProblem.addConstr( quicksum(1 - Igen[i,k] for k in range(j, j+int(self.genfile.loc[i,'minOFF']))) >= self.genfile.loc[i,'minOFF'] * Zgen[i,j], \
                                          'Gen MinON/OFF Constraint')  

        for j in hourlist:
            OriginalProblem.addConstr( GenCost[j] == quicksum(Ygen[i,j] * self.genfile.loc[i,'SUCost'] \
                                          + Zgen[i,j] * self.genfile.loc[i,'SDCost'] \
                                          + Igen[i,j] * self.genfile.loc[i,'NOLOADCOST'] * self.genfile.loc[i,'Pmin'] \
                                          + Pk1[i,j] * self.genfile.loc[i,'k1'] + Pk2[i,j] * self.genfile.loc[i,'k2'] \
                                          + Pk3[i,j] * self.genfile.loc[i,'k3'] + Pk4[i,j] * self.genfile.loc[i,'k4'] for i in nonreslist), \
                                       'Generation Cost Constraint')
    
            OriginalProblem.addConstr( quicksum(Pgen[g,j] for g in genlist) == self.loadfile.loc[j,'all'], \
                                'Power Balance Constraint') 
    
            for i in linelist:
                OriginalProblem.addConstr( quicksum(self.PTDFfile.loc[i,self.genfile.loc[g,'busname']] * Pgen[g,j] for g in genlist) \
                                  - quicksum(self.PTDFfile.loc[i,d] * self.loadfile.loc[j,d] for d in buslist) <= self.linefile.loc[i,'FlowLim'], \
                                          'DC Power Flow Constraint') 
                OriginalProblem.addConstr( quicksum(self.PTDFfile.loc[i,self.genfile.loc[g,'busname']] * Pgen[g,j] for g in genlist) \
                                  - quicksum(self.PTDFfile.loc[i,d] * self.loadfile.loc[j,d] for d in buslist) >= -self.linefile.loc[i,'FlowLim'], \
                                          'DC Power Flow Constraint')  
        
        OriginalProblem.setObjective( quicksum(GenCost[j] for j in hourlist), GRB.MINIMIZE)
        
        OriginalProblem.update()
        
        return OriginalProblem

In [3]:
LPModel = Create_UC_Problem("", "gen.csv", "line.csv", "load.csv", "bus.csv", "res.csv", "PTDF.csv", "Origin").create_instance()

Academic license - for non-commercial use only


In [4]:
LPModel

<gurobi.Model MIP instance Origin: 1752 constrs, 816 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>

In [4]:
LPModel.setParam(GRB.Param.Threads, 8)
LPModel.optimize()

Changed value of parameter Threads to 8
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 1752 rows, 816 columns and 6573 nonzeros
Variable types: 528 continuous, 288 integer (288 binary)
Coefficient statistics:
  Matrix range     [5e-03, 6e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 537821.87671
Presolve removed 1748 rows and 811 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 17 nonzeros
Found heuristic solution: objective 447012.41407
Variable types: 5 continuous, 0 integer (0 binary)

Root relaxation: objective 4.385142e+05, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    438514.24749 438514.247  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 avail

In [7]:
class Animal(object):
    def run(self):
        print('running...')

class Dog(Animal):
    def run(self):
        print("Dog running...")

class Cat(Animal):
    def run(self):
        print("Cat running...")

In [10]:
dog1 = Animal()

In [11]:
dog1.run()

running...


In [12]:
def run_twice(animal):
    animal.run()
    animal.run()

In [13]:
run_twice(Animal())

running...
running...


In [14]:
run_twice(Dog())

Dog running...
Dog running...


In [15]:
class Tortoise(Animal):
    pass

In [16]:
run_twice(Tortoise())

running...
running...


In [21]:
type(Dog())

__main__.Dog

In [22]:
'ABC'.__len__()

3